In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import os 

from utils_models.utils_functions import canonical_form, normalize_features, get_model_matrices, standard_form1, build_model_from_json, \
print_model_in_mathematical_format

In [2]:
project_root = os.path.dirname(os.getcwd())
problem = os.path.join(project_root, 'data/GAMS_library', 'SENSTRAN_modified.mps')
os.path.exists(problem)

True

In [3]:
model = gp.read(problem)
model.optimize()
# original_primal, track_elements = canonical_form(model)
# print(type(original_primal))
# original_primal.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-11
Read MPS format model from file /home/brgonzalez/Escritorio/TESIS/Code/Interpretable_Optimization/data/GAMS_library/SENSTRAN_modified.mps
Reading time = 0.00 seconds
Convert: 5 rows, 6 columns, 12 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x27bd806d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.000000e+02   0.000000e+00      0s
       5    1.6398000

In [4]:
if model.Status == GRB.OPTIMAL:
    print('Optimal objective value:', model.ObjVal)
    # Get the variable names and values
    variables = model.getVars()
    var_names = [v.VarName for v in variables]
    var_values = model.getAttr('X', variables)
    # Iterate over names and values using zip
    for var_name, value in zip(var_names, var_values):
        print(f'{var_name}: {value}')
elif model.Status == GRB.INFEASIBLE:
    print('Model is infeasible.')
elif model.Status == GRB.UNBOUNDED:
    print('Model is unbounded.')
else:
    print('Optimization ended with status %d' % model.Status)


Optimal objective value: 163.98
x1: 0.0
x2: 300.0
x3: 50.0
x4: 325.0
x5: 0.0
x6: 225.0


Resolve normalize

In [5]:
def build_normalized_model(A_norm, b_norm, c, lb, ub, of_sense, cons_senses, var_names):
    """
    Builds a Gurobi model using the normalized A and b, and original c, lb, ub.
    """
    model = gp.Model()

    # Set objective sense
    model.ModelSense = of_sense  # 1 for minimization, -1 for maximization

    # Add variables
    num_vars = len(c)
    variables = []
    for i in range(num_vars):
        var = model.addVar(lb=lb[i], ub=ub[i], obj=c[i], name=var_names[i])
        variables.append(var)
    model.update()

    # Add constraints
    num_constrs = len(b_norm)
    for i in range(num_constrs):
        expr = gp.LinExpr()
        for j in range(num_vars):
            if A_norm[i, j] != 0:
                expr.add(variables[j], A_norm[i, j])
        sense = cons_senses[i]
        rhs = b_norm[i]
        if sense == '<':
            model.addConstr(expr <= rhs)
        elif sense == '>':
            model.addConstr(expr >= rhs)
        elif sense == '=':
            model.addConstr(expr == rhs)
        else:
            raise ValueError(f"Unknown constraint sense: {sense}")
    model.update()

    return model


In [6]:
A, b, c, co, lb, ub, of_sense, cons_senses, variables_names = get_model_matrices(original_primal)
A_norm, b_norm, scalers = normalize_features(A, b) 
# Run the model with normalized features
normalized_model = build_normalized_model(A_norm, b_norm, c, lb, ub, of_sense, cons_senses, variables_names)

# Optimize the normalized model
normalized_model.optimize()

# Retrieve and print results
if normalized_model.Status == GRB.OPTIMAL:
    print('Optimal objective value (normalized model):', normalized_model.ObjVal)
    solution = normalized_model.getAttr('X', normalized_model.getVars())
    for var_name, value in zip(variables_names, solution):
        print(f'{var_name}: {value}')
else:
    print('Optimization was not successful.')

# # Optional: Compare with the original model
# model.optimize()
# if model.Status == GRB.OPTIMAL:
#     print('Optimal objective value (original model):', model.ObjVal)
# else:
#     print('Original model optimization was not successful.')


NameError: name 'original_primal' is not defined

In [ ]:
standard_model = standard_form1(model)
standard_model.optimize()
if standard_model.Status == GRB.OPTIMAL:
    print('Optimal objective value (standard model):', standard_model.ObjVal)
    solution = standard_model.getAttr('X', standard_model.getVars())
    # for var_name, value in zip(variables_names, solution):
    #     print(f'{var_name}: {value}')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 5 rows, 11 columns and 17 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6398000e+02   5.000000e+01   0.000000e+00      0s
       1    1.6398000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.639800000e+02
Optimal objective value (standard model): 163.98


In [ ]:
GAMS_path = os.path.join(project_root, 'data/GAMS_library')
for model_name in os.listdir(GAMS_path):
    if model_name.endswith('.xlsx'):
        continue
    model_path = os.path.join(GAMS_path, model_name)
model = gp.read(problem)
# Solve the problem in canonical form and standard form
original_primal, track_elements = canonical_form(model)
standard_model = standard_form1(model)
print_model_in_mathematical_format(standard_model)
original_primal.optimize()
standard_model.optimize()
# Check if the objective values are the same
# modle status
if original_primal.Status == GRB.OPTIMAL and standard_model.Status == GRB.OPTIMAL:
    print('Optimal objective value (original model):', original_primal.ObjVal)
    print('Optimal objective value (standard model):', standard_model.ObjVal)
# print the value of the variables
    for var in original_primal.getVars():
        print(f'{var.VarName}: {var.X}')
    for var in standard_model.getVars():
        print(f'{var.VarName}: {var.X}')

Read MPS format model from file /home/brgonzalez/Escritorio/TESIS/Code/Interpretable_Optimization/data/GAMS_library/SENSTRAN_modified.mps
Reading time = 0.00 seconds
Convert: 5 rows, 6 columns, 12 nonzeros
Minimize
0.225 x1 + 0.153 x2 + 0.162 x3 + 0.225 x4 + 0.162 x5 + 0.1638 x6

Subject To
x1 + x2 + x3 + slack_e2 = 350.0
x4 + x5 + x6 + slack_e3 = 600.0
x1 + x4 + -1.0 slack_e4 = 325.0
x2 + x5 + -1.0 slack_e5 = 300.0
x3 + x6 + -1.0 slack_e6 = 275.0

Bounds
0.0 <= x1 <= +inf
0.0 <= x2 <= +inf
0.0 <= x3 <= +inf
0.0 <= x4 <= +inf
0.0 <= x5 <= +inf
0.0 <= x6 <= +inf
0.0 <= slack_e2 <= +inf
0.0 <= slack_e3 <= +inf
0.0 <= slack_e4 <= +inf
0.0 <= slack_e5 <= +inf
0.0 <= slack_e6 <= +inf

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model finge

In [12]:
# Load model from json 
path = os.path.join(os.getcwd(), 'model_matrices.json')   
model = build_model_from_json(path)
print_model_in_mathematical_format(model)
model.optimize()
# Calculate the value of each variable
# variables = model.getVars()
# var_names = [v.VarName for v in variables]
# var_values = model.getAttr('X', variables)
# # Iterate over names and values using zip
# for var_name, value in zip(var_names, var_values):
#     print(f'{var_name}: {value}')

Minimize
2.6 x23 + 0.65 x24 + 4.55 x25 + 2.6 x26 + 2.6 x27 + 3.9 x28 + 9.1 x29 + 0.7 x33 + 1.4 x34 + 2.8 x35 + 1.4 x36 + 0.7 x37 + 1.4 x38 + 1.4 x39 + 2.1 x40 + 1.4 x41 + 0.7 x42 + 0.1 x43 + 0.8 x44 + 0.1 x45 + x49_pos + -1.0 x49_neg

Subject To
x1 + x2 + x3 + x4 + x5 + slack_e1 = 10.0
x6 + x7 + x8 + x9 + slack_e2 = 19.0
x10 + x11 + x12 + slack_e3 = 25.0
x13 + x14 + x15 + x16 + x17 + slack_e4 = 15.0
-16.0 x1 + -9.0 x13 + x18 = 0.0
-15.0 x2 + -10.0 x6 + -5.0 x10 + -11.0 x14 + x19 = 0.0
-28.0 x3 + -14.0 x7 + -22.0 x15 + x20 = 0.0
-23.0 x4 + -15.0 x8 + -7.0 x11 + -17.0 x16 + x21 = 0.0
-81.0 x5 + -57.0 x9 + -29.0 x12 + -55.0 x17 + x22 = 0.0
-1.0 x18 + -1.0 x23 + slack_e10 = -200.0
-1.0 x18 + -1.0 x24 + slack_e11 = -220.0
-1.0 x18 + -1.0 x25 + slack_e12 = -250.0
-1.0 x18 + -1.0 x26 + slack_e13 = -270.0
-1.0 x18 + -1.0 x27 + slack_e14 = -300.0
-1.0 x19 + -1.0 x28 + slack_e15 = -50.0
-1.0 x19 + -1.0 x29 + slack_e16 = -150.0
-1.0 x19 + -1.0 x30 + slack_e17 = 0.0
-1.0 x19 + -1.0 x31 + slack_e18